<a href="https://colab.research.google.com/github/pieva/SimPy/blob/main/Simpy_Example_%E2%80%93_Jackson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://planet.racket-lang.org/package-source/williams/simulation.plt/3/2/planet-docs/simulation/simpy.html#(part._.Jackson)

In [13]:
!pip install simpy

import simpy
from random import Random, expovariate, uniform

'''
Simulation of a Jackson queue network

This code simulates a Jackson queue network using the simpy library in Python.
A Jackson queue network is a type of queueing network where each node has an exponential service time.
Messages (customers) can move between nodes based on a transition probability matrix.

'''

#Variables to store statistics about the simulation:
time_in_system = [] # mean delay in the system
no_in_system = []   # mean number in the system
stime = []          # service time (stime)

def sum(P):
    """ Sum of the real vector P """
    sumP = 0.0
    for i in range(len(P)):
        sumP += P[i]
    return sumP


def choose2dA(i, P, g):
    """  return a random choice from a set j = 0..n-1
       with probs held in list of lists P[j] (n by n)
       using row i
       g = random variable
       call:  next = choose2d(i,P,g)
    """
    U = g.random()
    sumP = 0.0
    for j in range(len(P[i])):  # j = 0..n-1
        sumP += P[i][j]
        if U < sumP:
            break
    return j


'''
Msg Class:
This class represents a message (customer) in the queue network.
When a message arrives at a node, it gets serviced and may move to another node based on the transition probabilities defined in the matrix P.
The execute method simulates the message's behavior in the system.
'''
class Msg(object):
    def __init__(self, i, node):
        self.i = i
        self.node = node

    def execute(self, env, processor):
        """ executing a message """
        global noInSystem
        startTime = env.now
        noInSystem += 1
        no_in_system.append(noInSystem)
        ##print "DEBUG noInSystm = ",noInSystem
        self.trace(env, "Arrived node  %d" % (self.node,))
        while self.node != 3:
            with processor[self.node].request() as req:
                yield req
                self.trace(env, "Got processor %d" % (self.node,))
                st = Mrv.expovariate(1.0 / mean[self.node])
                stime.append(st)
                yield env.timeout(st)
                self.trace(env, "Finished with %d" % (self.node,))
            self.node = choose2dA(self.node, P, Mrv)
            self.trace(env, "Transfer to   %d" % (self.node,))
        time_in_system.append(env.now - startTime)
        self.trace(env, "leaving       %d" % (self.node,), noInSystem)
        noInSystem -= 1

    def trace(self, env, message, nn=0):
        if MTRACING:
            print("%7.4f %3d %10s %3d" % (env.now, self.i, message, nn))


'''
Generator Class:
This class generates a sequence of messages at a given arrival rate (rate) and schedules them for execution in the Msg class.
The execute method generates messages until either the maximum number of messages (maxNumber) or the maximum time (maxTime) is reached.
'''
class Generator(object):
    def __init__(self, env, rate, maxT, maxN):
        self.env = env
        self.name = "Generator"
        self.rate = rate
        self.maxN = maxN
        self.maxT = maxT
        self.g = Random(11335577)
        self.i = 0

    def execute(self, processor):
        while (self.env.now < self.maxT) & (self.i < self.maxN):
            self.i += 1
            p = Msg(self.i, startNode)
            env.process(p.execute(env, processor))
            ## self.trace("starting "+p.name)
            yield env.timeout(self.g.expovariate(self.rate))
        self.trace("generator finished with " + str(self.i) + " ========")

    def trace(self, message):
        if GTRACING:
            print("%7.4f \t%s" % (env.now, message))


# generator:
GTRACING = 0
# messages:
rate = 0.5 # arrival rate
noInSystem = 0
MTRACING = 0
startNode = 0
maxNumber = 10000
maxTime = 100000.0
Mrv = Random(77777)

# Mean service times
mean = [1.0, 2.0, 1.0]

# Transition probability matrix
P = [[0, 0.5, 0.5, 0],
     [0, 0, 0.8, 0.2],
     [0.2, 0, 0, 0.8]]

'''
Simulation: The main part of the code
- creates the simulation environment (env) using simpy.Environment(),
- initializes the processor resources (simpy.Resource),
- starts the simulation by calling env.process(g.execute(processor))
- the env.run(until=100.0) function runs the simulation until the time limit of 100.0 units.
'''
env = simpy.Environment()
processor = [simpy.Resource(env), simpy.Resource(env), simpy.Resource(env)]
g = Generator(env, rate,maxTime,maxNumber)
env.process(g.execute(processor))
env.run(until=100.0)

'''
Calculating Statistics:
After the simulation is complete, the code calculates various statistics, such as
- total number of jobs arrived and completed,
- total simulation time,
- mean arrival rate.
'''
mean_number = sum(no_in_system) / len(no_in_system)
mean_delay = sum(time_in_system) / len(time_in_system)
mean_stime = sum(stime) / len(stime)
total_jobs = len(time_in_system)
total_time = env.now
mean_rate = total_jobs / total_time

# It then prints out these statistics.
print("Mean numberm= %8.4f" % (mean_number,))
print("Mean delay  = %8.4f" % (mean_delay,))
print("Mean stime  = %8.4f" % (mean_stime,))
print("Total jobs  = %8d" % (total_jobs,))
print("Total time  = %8.4f" % (total_time,))
print("Mean rate   = %8.4f" % (mean_rate,))



Mean numberm=   5.5849
Mean delay  =   8.7237
Mean stime  =   1.2096
Total jobs  =       52
Total time  = 100.0000
Mean rate   =   0.5200
